In this code, we will be doing the evaluation of:

Load past predictions from our stored pickle files

Fetch actual match results from our database

Calculate accuracy metrics for each prediction type (match result, over/under, cards)

Analyze performance by league, team, home/away, and other factors

Visualize the results to identify patterns and strengths/weaknesses

In [ ]:
# 002_model_evaluation.ipynb - Evaluation Framework

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Connection to the database
conn = sqlite3.connect('db_sportmonks.db')

# 1. Function to load stored predictions
def load_predictions(file_path):
    """Load stored predictions from a pickle file"""
    if not os.path.exists(file_path):
        print(f"No predictions file found at {file_path}")
        return None
        
    with open(file_path, 'rb') as f:
        predictions = pickle.load(f)
    
    print(f"Loaded {len(predictions)} predictions from {file_path}")
    return predictions

# 2. Function to get actual match results
def get_match_results(conn, fixture_ids):
    """Get actual results for a list of fixture IDs"""
    if not fixture_ids:
        return pd.DataFrame()
    
    # Create a comma-separated string of fixture IDs
    fixture_ids_str = ','.join(map(str, fixture_ids))
    
    # Query to get match results
    query = f"""
    SELECT 
        f.id as fixture_id,
        f.league_id,
        f.home_team_id,
        f.away_team_id,
        f.score_home,
        f.score_away,
        f.starting_at,
        ht.name as home_team,
        at.name as away_team
    FROM 
        fixtures f
    JOIN 
        teams ht ON f.home_team_id = ht.id
    JOIN 
        teams at ON f.away_team_id = at.id
    WHERE 
        f.id IN ({fixture_ids_str})
        AND f.score_home IS NOT NULL
        AND f.score_away IS NOT NULL
    """
    
    # Execute query
    results_df = pd.read_sql_query(query, conn)
    
    if results_df.empty:
        print("No match results found for the provided fixture IDs")
        return results_df
    
    # Add calculated fields
    results_df['total_goals'] = results_df['score_home'] + results_df['score_away']
    results_df['result'] = np.where(results_df['score_home'] > results_df['score_away'], 'Home Win',
                          np.where(results_df['score_home'] == results_df['score_away'], 'Draw', 'Away Win'))
    results_df['over_under'] = np.where(results_df['total_goals'] > 2.5, 'Over', 'Under')
    
    # Try to get card data if available
    try:
        # Create a comma-separated string of fixture IDs
        cards_query = f"""
        SELECT 
            fixture_id,
            COUNT(*) as total_cards
        FROM 
            cards
        WHERE 
            fixture_id IN ({fixture_ids_str})
        GROUP BY 
            fixture_id
        """
        
        cards_df = pd.read_sql_query(cards_query, conn)
        
        # Merge with results
        if not cards_df.empty:
            results_df = results_df.merge(cards_df, on='fixture_id', how='left')
            results_df['total_cards'] = results_df['total_cards'].fillna(0)
            
            # Determine cards over/under threshold (you may want to adjust this)
            card_threshold = 4.5  # Example threshold
            results_df['cards_over_under'] = np.where(results_df['total_cards'] > card_threshold, 'Over', 'Under')
        else:
            results_df['total_cards'] = np.nan
            results_df['cards_over_under'] = np.nan
    except Exception as e:
        print(f"Error getting card data: {e}")
        results_df['total_cards'] = np.nan
        results_df['cards_over_under'] = np.nan
    
    return results_df

# 3. Function to calculate accuracy metrics
def calculate_metrics(predictions, results):
    """Calculate evaluation metrics for predictions"""
    if predictions is None or results.empty:
        print("No predictions or results to evaluate")
        return None
    
    # Convert results DataFrame to dictionary for easier access
    results_dict = results.set_index('fixture_id').to_dict(orient='index')
    
    # Initialize counters
    metrics = {
        'result': {'correct': 0, 'total': 0},
        'over_under': {'correct': 0, 'total': 0},
        'cards': {'correct': 0, 'total': 0}
    }
    
    # Evaluate each prediction
    for pred in predictions:
        fixture_id = pred['fixture_id']
        
        # Skip if we don't have actual results for this fixture
        if fixture_id not in results_dict:
            continue
        
        actual = results_dict[fixture_id]
        
        # Match result evaluation
        if 'result' in pred['predictions'] and 'prediction' in pred['predictions']['result']:
            predicted_result = pred['predictions']['result']['prediction']
            actual_result = actual['result']
            
            metrics['result']['total'] += 1
            if predicted_result == actual_result:
                metrics['result']['correct'] += 1
        
        # Over/Under evaluation
        if 'over_under' in pred['predictions'] and 'prediction' in pred['predictions']['over_under']:
            predicted_ou = pred['predictions']['over_under']['prediction']
            actual_ou = actual['over_under']
            
            metrics['over_under']['total'] += 1
            if predicted_ou == actual_ou:
                metrics['over_under']['correct'] += 1
        
        # Cards evaluation (if data available)
        if 'cards_over_under' in actual and actual['cards_over_under'] is not np.nan:
            if 'cards' in pred['predictions'] and 'prediction' in pred['predictions']['cards']:
                predicted_cards = pred['predictions']['cards']['prediction']
                actual_cards = actual['cards_over_under']
                
                metrics['cards']['total'] += 1
                if predicted_cards == actual_cards:
                    metrics['cards']['correct'] += 1
    
    # Calculate accuracy percentages
    for key in metrics:
        if metrics[key]['total'] > 0:
            metrics[key]['accuracy'] = metrics[key]['correct'] / metrics[key]['total']
        else:
            metrics[key]['accuracy'] = None
    
    return metrics

# 4. Function to calculate ROI for a betting strategy
def calculate_roi(predictions, results, stake=1.0, odds=None):
    """
    Calculate Return on Investment (ROI) for predictions
    
    Parameters:
    predictions - List of prediction dictionaries
    results - DataFrame with actual results
    stake - Amount to bet on each prediction (default: 1.0)
    odds - DataFrame with bookmaker odds (optional)
    
    Returns:
    Dictionary with ROI metrics
    """
    if predictions is None or results.empty:
        print("No predictions or results to evaluate")
        return None
    
    # Convert results DataFrame to dictionary for easier access
    results_dict = results.set_index('fixture_id').to_dict(orient='index')
    
    # Initialize totals
    total_investment = 0.0
    total_return = 0.0
    
    # If no odds provided, use default odds
    default_odds = {
        'Home Win': 2.5,
        'Draw': 3.5,
        'Away Win': 2.5,
        'Over': 2.0,
        'Under': 2.0
    }
    
    # Process each prediction
    wins = 0
    losses = 0
    
    for pred in predictions:
        fixture_id = pred['fixture_id']
        
        # Skip if we don't have actual results for this fixture
        if fixture_id not in results_dict:
            continue
        
        actual = results_dict[fixture_id]
        
        # Only consider match result for now
        if 'result' in pred['predictions'] and 'prediction' in pred['predictions']['result']:
            predicted_result = pred['predictions']['result']['prediction']
            actual_result = actual['result']
            
            # Determine outcome
            outcome_correct = (predicted_result == actual_result)
            
            # Get odds for this prediction
            if odds is not None and fixture_id in odds.index:
                if predicted_result == 'Home Win':
                    bet_odds = odds.loc[fixture_id, 'home_odds']
                elif predicted_result == 'Draw':
                    bet_odds = odds.loc[fixture_id, 'draw_odds']
                else:  # Away Win
                    bet_odds = odds.loc[fixture_id, 'away_odds']
            else:
                bet_odds = default_odds[predicted_result]
            
            # Calculate investment and return
            total_investment += stake
            
            if outcome_correct:
                total_return += stake * bet_odds
                wins += 1
            else:
                losses += 1
    
    # Calculate ROI
    if total_investment > 0:
        roi = (total_return - total_investment) / total_investment
    else:
        roi = 0.0
    
    return {
        'total_investment': total_investment,
        'total_return': total_return,
        'net_profit': total_return - total_investment,
        'roi': roi,
        'wins': wins,
        'losses': losses,
        'win_rate': wins / (wins + losses) if (wins + losses) > 0 else 0.0
    }

# 5. Check for any available results from our predictions
premier_league_predictions = load_predictions('premier_league_predictions.pkl')

if premier_league_predictions:
    # Extract fixture IDs
    fixture_ids = [pred['fixture_id'] for pred in premier_league_predictions]
    
    # Get actual results (if any are available)
    actual_results = get_match_results(conn, fixture_ids)
    
    if not actual_results.empty:
        print(f"Found {len(actual_results)} matches with results")
        
        # Calculate metrics
        metrics = calculate_metrics(premier_league_predictions, actual_results)
        
        if metrics:
            print("\nPrediction Accuracy:")
            for key, value in metrics.items():
                if value['accuracy'] is not None:
                    print(f"{key.capitalize()}: {value['correct']}/{value['total']} = {value['accuracy']:.2%}")
            
            # Calculate ROI
            roi_metrics = calculate_roi(premier_league_predictions, actual_results)
            
            if roi_metrics:
                print("\nBetting Performance:")
                print(f"Total Bets: {roi_metrics['wins'] + roi_metrics['losses']}")
                print(f"Wins: {roi_metrics['wins']}")
                print(f"Losses: {roi_metrics['losses']}")
                print(f"Win Rate: {roi_metrics['win_rate']:.2%}")
                print(f"Investment: ${roi_metrics['total_investment']:.2f}")
                print(f"Return: ${roi_metrics['total_return']:.2f}")
                print(f"Profit: ${roi_metrics['net_profit']:.2f}")
                print(f"ROI: {roi_metrics['roi']:.2%}")
        
    else:
        print("No results available yet for the predicted matches")
        print("Predictions were made for the following fixtures:")
        for pred in premier_league_predictions:
            print(f"  {pred['match']} on {pred['match_date']}")
else:
    print("No predictions found. Run the prediction notebook first.")

# Close the connection
conn.close()

here 